In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

In [27]:
# InfluxDB에서 데이터 로드 및 전처리 함수
def load_and_preprocess_data():
    # InfluxDB 연결 설정
    bucket = "elec"
    org = "ATG"
    token = "-Csx0nZp8yGSI5XlE6uVntoDntjN2j8-BXKr9FNuXfEHPINLioLag6nYUSPqmEu1-oTo_Q59ElgTsNH-TNwdnA=="
    url = "http://localhost:8086"
    client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
    query_api = client.query_api()

    # 데이터 쿼리 (1시간 간격으로 리샘플링)
    query = '''
    from(bucket: "electricity")
      |> range(start: 2012-04-01T00:00:00Z, stop: 2013-04-01T00:00:00Z)
      |> filter(fn: (r) => r["_measurement"] == "Electricity_B2E.csv_measurements")
      |> filter(fn: (r) => r["_field"] == "active_power" or r["_field"] == "apparent_power" or r["_field"] == "reactive_power")
      |> aggregateWindow(every: 10m, fn: mean, createEmpty: false)
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    result = query_api.query_data_frame(query=query, org=org)

    # 결과를 pandas DataFrame으로 변환
    df = result.set_index('_time')
    df = df[['active_power', 'reactive_power']]
    df.columns = ['P', 'Q']  # 열 이름 변경

    return df

In [10]:
# 시퀀스 데이터 생성 함수 (배치 처리)
def create_sequences(data, seq_length, batch_size=1000):
    n_samples = len(data) - seq_length
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        X = np.array([data[i:i+seq_length] for i in range(start_idx, end_idx)])
        y = np.array([data[i+seq_length] for i in range(start_idx, end_idx)])
        yield X, y

In [12]:
# 데이터 스케일링 함수
def scale_data(data):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

In [14]:
# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [16]:
# 모델 학습 함수
def train_model(model, train_data, val_data, criterion, optimizer, num_epochs, seq_length, batch_size):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in create_sequences(train_data, seq_length, batch_size):
            X_batch = torch.FloatTensor(X_batch)
            y_batch = torch.FloatTensor(y_batch)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in create_sequences(val_data, seq_length, batch_size):
                X_batch = torch.FloatTensor(X_batch)
                y_batch = torch.FloatTensor(y_batch)
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

In [18]:
# 예측 함수
def predict(model, data, seq_length, scaler):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(30 * 24):  # 30일 * 24시간
            seq = torch.FloatTensor(data[i:i+seq_length]).unsqueeze(0)
            pred = model(seq)
            predictions.append(pred.squeeze().numpy())
    return scaler.inverse_transform(np.array(predictions))

In [29]:
# 데이터 로드 및 전처리
df = load_and_preprocess_data()

# 데이터 스케일링
scaled_data, scaler = scale_data(df.values)

# 데이터 분할 (64% 학습, 16% 검증, 20% 테스트)
train_size = int(0.64 * len(scaled_data))
val_size = int(0.16 * len(scaled_data))
train_data = scaled_data[:train_size]
val_data = scaled_data[train_size:train_size+val_size]
test_data = scaled_data[train_size+val_size:]

# 모델, 손실 함수, 옵티마이저 정의
input_size = df.shape[1]
hidden_size = 50
num_layers = 2
output_size = df.shape[1]
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
### drop out
# 하이퍼파라미터 설정
seq_length = 7 * 24  # 1주일 (7일 * 24시간)
batch_size = 64
num_epochs = 50

In [ ]:
# 모델 학습
train_model(model, train_data, val_data, criterion, optimizer, num_epochs, seq_length, batch_size)

# 1개월치 예측
predictions = predict(model, test_data, seq_length, scaler)

Epoch [1/50], Train Loss: 0.3605, Val Loss: 0.1634


In [6]:
# 결과 시각화
plt.figure(figsize=(15, 5))
plt.plot(predictions[:, 0], label='Predicted Active Power')
plt.plot(predictions[:, 1], label='Predicted Reactive Power')
plt.plot(predictions[:, 2], label='Predicted Apparent Power')
plt.plot(scaled_data[:, 0], label='Predicted Active Power')
plt.plot(scaled_data[:, 1], label='Predicted Reactive Power')
plt.plot(scaled_data[:, 2], label='Predicted Apparent Power')
plt.legend()
plt.title('1 Month Power Prediction')
plt.xlabel('Time (hours)')
plt.ylabel('Power')
plt.show()

NameError: name 'predictions' is not defined

<Figure size 1500x500 with 0 Axes>

In [11]:
## MSE 비교 코드 추가하기
model.save()